# Scraping Data

We chose to use the data provided by the Crunchbase API as it gathers up-to-date information about hundred of thousands of companies and it's considered as the online start ups database reference. 

The API accessible on the following link https://data.crunchbase.com/ is well documented and we only needed to ask for a user key. However, we had to work hard on understanding its intern architecture and on how to manipulate it for our purposes.

The API stores classes corresponding to each object we are manipulating (Company, Locations, People...) but also for the relationship between them. After digging deeper into attributes accessible on the API, we found out that the best way to retrieve the data was by relationship on each company. That's why, for each company, we need to do several API calls, one for each relationship we consider. We then realized how important it was to make your code sleep in between the calls as the API put a cap on the authorized bandwith per user key.

First, we scrapped all the organization names and short description from the REST API in the list 'organizations_tot', which gave us 408 422 organizations and took 3h30. We had to sleep in between our API calls because the server could identify our API key and limit our call. This file is saved as a json in 'tmp/organizations_json.json', we mapped the company permalink (ie name in the REST API naming) with its index in the previous list for an easy access.

Second, we used the excel API to retrieve a list of companies names for which crunchbaes contains enough information. We realized indeed that there were too many missing values on the companies provided by the REST API so a first filtering was required; also it would take too much time to retrieve these information for all the companies because of the API cap on the calls. The Excel API provides a good one because all the companies listed there contain at least information about categories, offices location and funding information. More important is that it stores the fundation date. We chose to build models on the companies funded before 2010 to be sure to have an horizon large enough to determine its success. The excel API gave us a list of 61 399 companies we reduced to 25 000 with the filter on the foundation date. For time matters, we chose to retrieve **8 000 companies**. This list is stored in the file 'organizations_xlsx.json'

Third, we had to retrieve the features we wanted from the REST API. It is is built around a relationship graph between the different instance (company, person, funding...). We went through the different relationships and instance to store only the one we care about. This provided the list 'relationships', which stores the relationships we consider, and the dictionary 'relationships_infos' which stores for each instance type the attributes we consider. Once this preliminary work has been done, we wrote the multiple calls needed to store these attributes for each organization. The json is stored in 'organizations_dict.json'. It's a dictionary by permalink with attributes corresponding to the features we care about (some atributes are also dictionaries). 

* [Scraping Data](#.-Scraping-Data)
* [1. Organization List](#1.-Organization-List)
* [2. Excel API](#2.-Excel-API)
* [3. Relationships](#3.-Relationships) 


In [1]:
import requests
import time
import json

def get_json(url):
    r_ = requests.get(url)
    try:
        return r_.json()
    except ValueError:
        time.sleep(1)
        return get_json(url)

user_key = '1a49911e9216b8037dc430f811fd5b6b'

# 1 Organizations List

First, we expose how to call the API and the structure we care about.

In [2]:
r = requests.get('https://api.crunchbase.com/v/3/organizations?user_key={}'.format(user_key))

The list of companies is the value for the keys items of the data of the request. The information to retrieve is then in the values of the key 'properties'

In [3]:
r.json()['data']['items'][0]

{u'properties': {u'api_path': u'organizations/dcode-economic-and-financial-consulting',
  u'city_name': None,
  u'country_code': None,
  u'created_at': 1449800531,
  u'domain': u'dcodeefc.com',
  u'facebook_url': None,
  u'homepage_url': u'http://dcodeefc.com/',
  u'linkedin_url': None,
  u'name': u'Dcode Economic and Financial Consulting',
  u'permalink': u'dcode-economic-and-financial-consulting',
  u'primary_role': u'company',
  u'profile_image_url': u'http://public.crunchbase.com/t_api_images/v1449800495/yijj0vm1vxh0vsp7gmni.png',
  u'region_name': None,
  u'short_description': u'Dcode EFC is a growing economic and financial consulting firm currently operating in Egypt.',
  u'twitter_url': None,
  u'updated_at': 1449800559,
  u'web_path': u'organization/dcode-economic-and-financial-consulting'},
 u'type': u'OrganizationSummary',
 u'uuid': u'd0112f89fa19cba8e9f2d1bea73dc27d'}

We show here how to retrieve the companies from the enxt page as each page contains only 100 items. This would be done by a manual change of the 'next_page_url' until we reached the totatl number of pages

In [4]:
r.json()['data']['paging']

{u'current_page': 1,
 u'items_per_page': 100,
 u'next_page_url': u'https://api.crunchbase.com/v/3/organizations?page=2',
 u'number_of_pages': 4159,
 u'prev_page_url': None,
 u'sort_order': u'created_at DESC',
 u'total_items': 415865}

This is the main script of this part. We iterate over the pages and then append the list of item to the current organizations list. It took around **5 hours** to run (because of the API cap).

In [14]:
organizations = []
for i in xrange(1, 4087):
    link = 'https://api.crunchbase.com/v/3/organizations?page={}&user_key={}'.format(i, user_key)
    if i%100 == 0:
        print 'page ', i, ' length is ',len(organizations)
    r_json = get_json(link)
    organizations += r_json['data']['items']
    time.sleep(0.1)

with open('tmp/organizations_json.json', 'w') as f:
    json.dump(organizations, f)

page  3400  length is  3300
page  3500  length is  13300
page  3600  length is  23300
page  3700  length is  33300
page  3800  length is  43300
page  3900  length is  53300
page  4000  length is  63300


In [218]:
# map from the permalink of a company to its index in the list organizations_tot
permalink2index = {organizations_tot[i]['properties']['permalink']: i for i in xrange(len(organizations_tot))}

We retrieved in total 400 000 companies and stored a mapping from the permalink to the index in the organisations list. (The cells results are not the global ones because we needed to rerun some of them because of API calls error)

# 2 Excel API

The excel companies spreadsheet was retrieved directly from the website of crunchbase API. We load it with the python library openpyxl and iterate through it to build the list of companies.

In [6]:
from openpyxl import load_workbook
wb = load_workbook(filename='test.xlsx', read_only=True)

/usr/local/lib/python2.7/site-packages/openpyxl/workbook/names/named_range.py:124: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")


We go into the sheet 'Companies' of the excel file and retrieve only the company name with tis founded date.

This is the main script of this part where we retrieve the list of companies from the excel API as a dictionary under the format 'permalink' : 'founded month'

In [9]:
wc = wb['Companies']
# List of the columns names to retrieve the index of the columns we care about
rows = enumerate(wc.rows)

# Removing the first element because it is not in the good format
row = rows.next()
for i, val in enumerate(row[1]):
    if val.value == 'founded_month':
        month_col = i
    if val.value == 'permalink':
        permalink_col = i

In [10]:
# Dictionary with (key, value) = (permalink, founded month)
organizations_xlsx = {}
rows = enumerate(wc.rows)

# Removing the header of the generator
row = rows.next()

for row in rows:
    # Get rid of the first 14 characters which are '/organizations/'
    organizations_xlsx[row[1][permalink_col].value[14:]] = row[1][month_col].value
len(organizations_xlsx)

61398

In [ ]:
# Saving the list
# with open('tmp/organizations_xlsx.json', 'w') as f:
#   json.dump(organizations_xlsx, f)
# Loading the list (not to run again the code above)
with open('tmp/organizations_xlsx.json', 'r') as f:
    organizations_xlsx = json.load(f)

In [11]:
# Retrieving the list of the company founded before 2011
import collections

# This counter counts the number of companies founded for each month
companies_years = collections.Counter(organizations_xlsx.values())
print 'Number of companies funded before 2011: ',sum([val for k, val in companies_years.iteritems() if k and int(k[:4]) < 2011])

# Dictionnary extracted from oraganizations_xlsx
focus_companies = {k:v for k,v in organizations_xlsx.iteritems() if v and int(v[:4]) < 2011}

Number of companies funded before 2011:  25171


# 3 Relationships

Here we retrieve from the crunchbase API the attributes we are interested in for each object type and the relationships we are going to fetch for each company.

In [12]:
relationships = ['founders', 'board_members_and_advisors', 'investors', 'headquarters', 'offices', 'categories', 'competitors', 'funding_rounds', 'acquisitions', 'acquired_by', 'ipo']

relationships_infos = {}
relationships_infos['person'] = ['bio', 'born_on', 'permalink']
relationships_infos['address'] = ['city', 'country']
relationships_infos['market'] = ['name']
relationships_infos['product'] = ['name']
relationships_infos['organization'] = ['permalink']
relationships_infos['fundinground'] = ['announced_on', 'funding_type', 'money_raised_usd']
relationships_infos['acquisition'] = ['announced_on']
relationships_infos['ipo'] = ['went_public_on']

In [13]:
# We are retrieving only the companies founded before 2011
len(focus_companies)

25171

This is the main script of this part. It extracts the features for each organization and requires several API calls per iteration. As a result, it takes a lot of time to run (around 5 hours for 1 000 companies).

We first load the current json dictionary and then append to it the new companies we retrieved. We chose this structure of implementation to be able to run it in multiple times given the execution time needed.

In [2]:
organizations_dict = {}
# First load the organizations we already have and start from there
with open('tmp/organizations_dict_to2010.json', 'r') as f:
    organizations_dict = json.load(f)
print len(organizations_dict)

6852


In [ ]:
%%time
for i, permalink in enumerate(focus_companies.keys()[len(organizations_dict):len(focus_companies)]):
    if i%50 ==0:
        print 'iteration ', i, ' length is ', len(organizations_dict)
    infos = {}
    #permalink = organization['properties']['permalink']
    for relation in relationships:
        time.sleep(0.1)
        url = u'https://api.crunchbase.com/v/3/organizations/{}/{}?user_key={}'.format(permalink, relation, user_key)
        r_json = get_json(url)
        if 'data' not in r_json:
            continue
        if 'items' in r_json['data']:
            items = r_json['data']['items']
        elif 'item' in r_json['data']:
            items = r_json['data']['item']
        else:
            items = []
        number_pages = r_json['data']['paging']['number_of_pages']
        next_page_url = r_json['data']['paging']['next_page_url']
        for j in xrange(2, number_pages + 1):
            url = next_page_url[:-1]+str(j)+'&user_key='+user_key
            r_json = get_json(url)
            items += r_json['data']['items']
        
        relation_items = []
        if type(items) is dict:
            items = [items]
        for item in items:
            if 'relationships' in item:
                for relationships_key in item['relationships']:
                    relationships_key = relationships_key.lower()
                    if relationships_key in relationships_infos:
                        item_temp = {}
                        for infos_key in relationships_infos[relationships_key]:
                            item_temp[infos_key] = item['relationships'][relationships_key]['properties'][infos_key]
                        relation_items.append(item_temp)
            relationships_key = item['type'].lower()
            if relationships_key in relationships_infos:
                item_temp = {}
                for infos_key in relationships_infos[relationships_key]:
                    item_temp[infos_key] = item['properties'][infos_key]
                relation_items.append(item_temp)
        infos[relation] = relation_items
    organizations_dict[permalink] = infos

    with open('tmp/organizations_dict_to2010.json', 'w') as f:
        json.dump(organizations_dict, f)

In [3]:
# This file is not present on the github repository because it's too heavy. 
with open('../organizations_total_json.json', 'r') as f:
    organizations_tot = json.load(f)

In [4]:
permalink2index = {v['properties']['permalink']:i for i, v in enumerate(organizations_tot)}
print len(permalink2index)
print len(organizations_tot)

407691
408422


In [6]:
# Adding the short description to the stored information (from the json with all the organizations)
for o in organizations_dict:
    if o in permalink2index:
        i = permalink2index[o]
        organizations_dict[o]['short_description'] = organizations_tot[i]['properties']['short_description']


In [7]:
# If the dictionary is not in memory yet
# with open('tmp/organizations_xlsx.json', 'r') as f:
#    organizations_xlsx = json.load(f)

In [8]:
# Adding the foundation date to the stored information (from the excel json)
for o in organizations_dict:
    if o in organizations_xlsx:
        organizations_dict[o]['month_founded'] = organizations_xlsx[o]


In [10]:
with open('tmp/organizations_dict_to2010.json', 'w') as f:
    json.dump(organizations_dict, f)

In [11]:
len(organizations_dict)

6852

As a conclusion, our main learning from the scraping is that even though an API is available, you still need to dig deeper into the systems design. Besides, scrapping a lot of information over the wire is not immediate, it can take a lot of time especially if many calls are required and a cap is put on the bandwith.

Building a robust and easy to use API is really important.
We'll keep that advice in mind for the companies we are going to invest in!